In [1]:
from __future__ import unicode_literals, print_function
import spacy
import plac
from spacy.lang.en import English
from spacy.matcher import PhraseMatcher
from spacy.tokens import Doc, Span, Token
import random
from pathlib import Path

nlp = spacy.load('en')
docx = nlp(u"<MTS Acquisition Corp>, a company formed by Martin T. Sosnoff, said it has started a tender offer for all shares of Caesars World Inc at 28 dlrs each. In a newspaper advertisement, MTS said the offer and withdrawal rights expire April Three unless extended. Sosnoff, a New York investor, already owns about four mln of Caesars\' 30.3 mln shares outstanding, or about 13.3 pct, and is Caesars\' largest shareholder. Caesars owns casino hotels in Nevada and honeymoon resorts in Pennsylvania\'s Pocono Mountains.")
for token in docx.ents:
    print(token.text,token.start_char,token.end_char,token.label_)
print(len("<MTS Acquisition Corp>, a company formed by Martin T. Sosnoff, said it has started a tender offer for all shares of Caesars World Inc at 28 dlrs each. In a newspaper advertisement, MTS said the offer and withdrawal rights expire April Three unless extended. Sosnoff, a New York investor, already owns about four mln of Caesars\' 30.3 mln shares outstanding, or about 13.3 pct, and is Caesars\' largest shareholder. Caesars owns casino hotels in Nevada and honeymoon resorts in Pennsylvania\'s Pocono Mountains."))

MTS Acquisition Corp 1 21 ORG
Martin T. Sosnoff 44 61 PERSON
Caesars World Inc 116 133 ORG
28 137 139 CARDINAL
MTS 181 184 ORG
April Three 229 240 DATE
Sosnoff 258 265 ORG
New York 269 277 GPE
about four 301 311 CARDINAL
Caesars 319 326 ORG
30.3 328 332 CARDINAL
about 13.3 360 370 CARDINAL
Caesars 383 390 ORG
Caesars 413 420 ORG
Nevada 443 449 GPE
Pennsylvania 475 487 GPE
Pocono Mountains 490 506 LOC
507


In [2]:
# training data
TRAIN_DATA = [
    ("USAir Group Inc said Piedmont Aviation Inc has agreed to be acquired for 69 dlrs per share. The company, in a newspaper advertisement, said it has started a tender offer for all Piedmont shares at that price, and the Piedmont board, with two directors absent, has unanimously approved the bid. The offer and withdrawal rights are to expire April Three unless extended, and the bid is to be followed by a merger at the same price. USAir said Piedmont has granted it an irrevocable option to buy up to 3,491,030 new shares under certain circumstances. Piedmont now has about 18.6 mln shares outstanding. USAir said the tender is conditioned on receipt of enough shares to give USAir at least a 50.1 pct interest in Piedmont on a fully diluted basis and approval by the U.S. Department of Transportation of a voting trust agreement permitting USAir to buy and hold shares pending review of its application to gain control of Piedmont. The company said its merger agreement with Piedmont provides that the offer is not to be amended without Piedmont's prior written consent in any way that would be adverse to Piedmont shareholders, but it said it could cut the number of shares to be bought without Piedmont's consent. USAir said it could reduce the number of Piedmont shares to be purchased in the offer to no less than the minimum number needed to cause the voting trust condition of the bid to be satisfied. In that case, it said if more than that minimum number of shares were tendered, it would buy shares on a pro rata basis. In February USAir had offered to pay 71 dlrs per share in cash for 50 pct of Piedmont's stock and 1.55 to 1.90 USAir shares for each remaining Piedmont share. Last week, Carl C. Icahn-controlled Trans World Airlines Inc <TWA> made a conditional offer to acquire USAir for 52 dlrs per share, a bid that was rejected by the USAir board. The Transportation Department on Friday rejected TWA's application to acquire USAir on the grounds that the application failed to comply with department regulations by omitting necessary information. TWA said it would refile today, providing the information needed. On Friday TWA said it had already acquired four mln shares or 15 pct of USAir. Reuter ", {
        'entities': [(0, 15, 'ORG'),(21,42,'ORG'),(73,75,'CARDINAL') ]
    }),
    ('<MTS Acquisition Corp>, a company formed by Martin T. Sosnoff, said it has started a tender offer for all shares of Caesars World Inc at 28 dlrs each. In a newspaper advertisement, MTS said the offer and withdrawal rights expire April Three unless extended. Sosnoff, a New York investor, already owns about four mln of Caesars\' 30.3 mln shares outstanding, or about 13.3 pct, and is Caesars\' largest shareholder. Caesars owns casino hotels in Nevada and honeymoon resorts in Pennsylvania\'s Pocono Mountains. It also controls Caesars New Jersey Inc <CJN>, which owns an Atlantic City, N.J., casino hotel. For the second quarter ended January 31, Caesars World earned 12.6 mln dlrs on revenues of 190.4 mln dlrs, up from earnings of 7,500,000 dlrs and revenues of 163.8 mln dlrs a year before. For all of fiscal 1986, the company earned 41.0 mln dlrs on revenues of 694.4 mln dlrs. MTS said the offer is conditioned on receipt of at least enough shares to give Sosnoff a majority interest on a fully diluted basis, the arrangement of sufficient financing to buy all Caesars shares not already owned and pay related costs and approval by the New Jersey Casino control Commission and the NEvada Gaming Commission and State Gaming Control Board. MTS said Marine Midland Banks Inc <MM> has committed to lend it 100 mln dlrs for the acquisition and use its best efforts to syndicate another 400 mln dlrs in senior financing for the transaction. It said its financial adviser, PaineWebber Group Inc <PWJ>, has stated in writing that subject to market conditions, it is highly confident that it can arrange commitments for up to 475 mln dlrs in "mezzanine" financing. MTS said it does not expect problems in obtaining New Jersey and Nevada regulatory approval for the acquisition, since ownership in a Caesars stake has already been cleared. In June 1986, Sosnoff requested a seat on the Caesars World board, a request that has not yet been granted. In September 1986, Sosnoff, who is chairman of <Atalanta/Sosnoff Capital Corp>, filed for clearance under U.S. antitrust laws to raise his interest in Caesars World to 25 pct. Sosnoff said, in a letter to Caesars World chairman and chief executive officer Henry Gluck, that "The decision to go directly to the shareholders was made at the urging of may financial and legal advisors, who repeatedly stressed to me the lack of responsiveness of the management in the past." Sosnoff, who said he has made numerous efforts to express his views to management on ways of maximizing shareholder values, said Caesars twicce refused his request for a board seat. "My advisers felt that, had I given you advance notice, you would have used the time to throw up obstacles to my offer rather than giving it serious consideration," he said. Sosnoff said he hopes that Caesars World management will be willing to negotiate an acquisition agreement with him. "As I have indicated publicly in the past, I believe operating management of the company has performed well and that appropriate consideration should be given to a significant equity interest for them in the company following the acquisition," Sosnoff said in the letter to Gluck. MTS said Sosnoff has asked the company to fix March 27 as the record date for the determination of shareholders entitled to authorize action without a meeting -- including the election or removal of directors. Reuter ', {
        'entities': [(1, 21, 'ORG'), (116, 133, 'ORG'),(137,139,'CARDINAL')]
    })
]

In [3]:
if 'ner' not in nlp.pipe_names:
    ner=nlp.create('ner')
    nlp.add_pipe(ner,last=True)
else:
    ner = nlp.get_pipe('ner')

In [ ]:


@plac.annotations(
    n_iter=("Number of training iterations", "option", "n", int))
def main(n_iter=5):
    
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
 #           print(losses)

    test1="Allegheny International Inc said it has entered into an agreement to\
        merge with an affiliate of First Boston Inc's <FPC> First Boston Corp in\
        a transaction valued at about 500 mln dlrs. Allegheny said the agreement \
        calls for holders of its common to receive 24.60 dlrs a share. Holders of\
        the company's 2.19 dlrs cumulative preference shares will receive 20 dlrs a share and\
        those owning its 11.25 dlrs convertible preferred will receive 87.50 dlrs a share.\
        Allegheny International said the agreement calls for the First Boston \
        affiliate to start a cash tender offer for all outstanding shares of \
        Allegheny's common, cumulative preferred and convertible preferred. The\
        company said First Boston has committed to provide all financing necessary\
        to acquire such securities in the tender offer which is estimated to be\
        about 500 mln dlrs. It said the agreement is also subject to the waiver\
        by March 13 of certain conditions under the company's existing bank credit\
        agreement. Allegheny International's statement did not name or describe the\
        First Boston affiliate involved in the agreement. It did say the offer for\
        the company's stock will be conditioned also on the valid tender of\
        securities representing at least a majority of the voting power for the\
        election of directors of the company and the valid tender of at least two\
        third's of the outstanding cumulative preferred and convertible preferred\
        shares. Reuter"
    
    doc = nlp(test1)
    print(len(doc.ents))

    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
#        print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])


if __name__ == '__main__':
    main()